Source: https://github.com/jeffheaton/app_deep_learning/blob/854a0e4a3982daea4e50d34e4d8fc0d9d806b960//t81_558_class_02_1_python_pandas.ipynb

Pandas is based on the dataframe concept found in the R programming language. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# display function provides a cleaner display than merely printing the data frame 
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 6)
df = pd.read_csv('data/auto-mpg.csv')
display(df)

,mpg,cylinders,displacement,...,year,origin,name
0,18.0,8,307.0,...,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,...,70,1,buick skylark 320
2,18.0,8,318.0,...,70,1,plymouth satellite
...,...,...,...,...,...,...,...
395,32.0,4,135.0,...,82,1,dodge rampage
396,28.0,4,120.0,...,82,1,ford ranger
397,31.0,4,119.0,...,82,1,chevy s-10


Generating a list of dictionaries with statistical information about the dataframe field-by-field (headers)

In [3]:
# strip non-numerics
df = df.select_dtypes(include=['int', 'float'])

headers = list(df.columns.values)
fields = []

for field in headers:
    fields.append(
        {
            "name": field,
            "mean": df[field].mean(),
            "var": df[field].var(),
            "sdev": df[field].std(),
        }
    )
    
for field in fields:
    print(field)

{'name': 'mpg', 'mean': 23.514572864321607, 'var': 61.089610774274405, 'sdev': 7.815984312565782}
{'name': 'cylinders', 'mean': 5.454773869346734, 'var': 2.893415439920003, 'sdev': 1.7010042445332119}
{'name': 'displacement', 'mean': 193.42587939698493, 'var': 10872.199152247384, 'sdev': 104.26983817119591}
{'name': 'weight', 'mean': 2970.424623115578, 'var': 717140.9905256763, 'sdev': 846.8417741973268}
{'name': 'acceleration', 'mean': 15.568090452261307, 'var': 7.604848233611383, 'sdev': 2.757688929812676}
{'name': 'year', 'mean': 76.01005025125629, 'var': 13.672442818627143, 'sdev': 3.697626646732623}
{'name': 'origin', 'mean': 1.5728643216080402, 'var': 0.6432920268850549, 'sdev': 0.8020548777266148}


The following code convert the list of dictionaries into a pd dataframe. To restore default pd display set display values to zero

In [4]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
df2 = pd.DataFrame(data=fields)
display(df2)

,name,mean,var,sdev
0,mpg,23.514573,61.089611,7.815984
1,cylinders,5.454774,2.893415,1.701004
2,displacement,193.425879,10872.199152,104.269838
3,weight,2970.424623,717140.990526,846.841774
4,acceleration,15.568090,7.604848,2.757689
5,year,76.010050,13.672443,3.697627
6,origin,1.572864,0.643292,0.802055


### Managing missing values

Missing values are a reality of machine learning. Every dataset has missing values. Most of the values are present in the MPG database. However, there are missing values in the horspower column. A coomon practice is to replace missing values with the median values for that column. The following code replaces any NA values in horsepower with the median.

In [5]:
df = pd.read_csv("data/auto-mpg.csv", na_values=["NA", "?"])

In [6]:
print(f"horsepower has na? {pd.isnull(df['horsepower']).values.any()}")

horsepower has na? True


In [7]:
print("Filling missing values...")
med = df['horsepower'].median()
df['horsepower'] = df['horsepower'].fillna(med)
# its common also don't fill with the median but drop the entire row with the missing value
# df = df.dropna()
print(f"horsepower has na? {pd.isnull(df['horsepower']).values.any()}")

Filling missing values...
horsepower has na? False


### Dealing with outliers

Outliers are values that are unusually high or low. We typically consider outliers to be a value that is several standard deviations from the mean. Sometimes outliers are simply errors; this is a result of observational error. Outliers can also be truly large or small values that may be difficult to adress. The following function can remove such values.

In [6]:
# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[
        (np.abs(df[name] - df[name].mean()) >= (sd*df[name].std()))
    ]
    df.drop(drop_rows, axis=0, inplace=True)

The code below will drop every row from the AutoMpg dataset where the horsepower is two standard deviations or more above of below the mean.

In [8]:
df = pd.read_csv("data/auto-mpg.csv", na_values=["NA", "?"])

# create feature vector 
med = df['horsepower'].median()
df['horsepower'] = df['horsepower'].fillna(med)

# drop the name column
df.drop(columns="name", axis=1, inplace=True)

# drop the outliers in horsepower
print(f"Length before MPG outliers dropped: {len(df)}")
remove_outliers(df, "mpg", 2)
print(f"Length after MPG outliers dropped: {len(df)}")

Length before MPG outliers dropped: 398
Length after MPG outliers dropped: 388


### Dropping Fields
Drop fields that are of no value for the neural networks training.

In [10]:
df = pd.read_csv("data/auto-mpg.csv", na_values=["NA", "?"])

print(f"Before drop: {list(df.columns)}")
df.drop("name", axis=1, inplace=True)
print(f"After drop: {list(df.columns)}")

Before drop: ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']
After drop: ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']


### Concatenating Rows and Columns
Python can concatenate rows and columns together to form new data frames. This code creates a new data frame from the name and horsepower columns in Auto MPG dataset

In [11]:
df = pd.read_csv("data/auto-mpg.csv", na_values=["NA", "?"])

col_horsepower=df['horsepower']
col_name = df['name']
result = pd.concat([col_name, col_horsepower], axis=1)

result.head()

,name,horsepower
0,chevrolet chevelle malibu,130.0
1,buick skylark 320,165.0
2,plymouth satellite,150.0
3,amc rebel sst,150.0
4,ford torino,140.0


The concat function can also concatenate rows together. This code concatenates the first two rows and the last two ros of the Auto MPG dataset.

In [12]:
# create a new dataframe from first 2 rows and last 2 rows
df = pd.read_csv("data/auto-mpg.csv", na_values=["NA", "?"])

result = pd.concat([df[0:2], df[-2:]], axis=0)
result.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
396,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger
397,31.0,4,119.0,82.0,2720,19.4,82,1,chevy s-10


### Training and Validation
We must evaluate a machine learning model based on its ability to predict values that it has never seen before. Because of this, we often divide the training data into a validation and training set. The machine learning model will learn from the training data but ultimately be evaluated based on the validation data.

The code below splits the MPG data into a training and validation set. The training set uses 80% of the data, and the validation set uses 20%. 

In [3]:
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", na_values=["NA", "?"])

# usually a god idea to shuffle
df = df.reindex(np.random.permutation(df.index))

mask = np.random.rand(len(df)) < 0.8
train_df = pd.DataFrame(df[mask])
validation_df = pd.DataFrame(df[~mask])

print(f'Training DF: {len(train_df)}')
print(f'Validation DF: {len(validation_df)}')

Training DF: 330
Validation DF: 68


### Converting a Dataframe to a Matrix
The dataframe values property can be used to convert the numerical data to numpy matrix

In [4]:
df[
    [
        "mpg",
        "cylinders",
        "horsepower",
        "weight",
        "acceleration",
        "year",
        "origin"
    ]
].values

array([[ 15. ,   8. , 145. , ...,  13. ,  73. ,   1. ],
       [ 16. ,   6. , 100. , ...,  18. ,  73. ,   1. ],
       [ 30.5,   4. ,  63. , ...,  17. ,  77. ,   1. ],
       ...,
       [ 31. ,   4. ,  68. , ...,  17.6,  82. ,   3. ],
       [ 14. ,   8. , 140. , ...,  16. ,  74. ,   1. ],
       [ 24.5,   4. ,  60. , ...,  22.1,  76. ,   1. ]])

### Managing categorial and continuous values
Neural networks require their input to be a fixed number of columns. This input format is very similar to spreadsheet data; it must be entirely numeric. It is essential to represent the data so that the neural network can train from it. The four basic types of data are:

- Character Data (strings)
- - Nominal: Individual discrete items, no order. For example, color, zip code, and shape.
- - Ordinal: Individual distinct items have an implied order. For example, grade level, job title, Starbucks(tm) coffe size (tall, vente, grande)
- Numeric Data
- - Interval: Numeric values, no defined start. For example, temperature.
- - Ratio: Numeric values, clearly defined start. For example, speed

### Encoding continous Values
One common transformation is to normalize the inputs. It is sometimes valuable to normalize numeric inputs in a standard form so that the program can easily compare these two values. Like percentages for simple matematical operations, in machine learning the z-score is a common way to normalize contininuus data in machine learning, and its defined as:

$$
z = \frac{x - \mu}{\sigma}
$$

where $\mu$ is the mean $\bar{x} = \frac{x_1+x_2+\cdots +x_n}{n}$

and $\sigma$ is the standard deviation  $= \sqrt{\frac{1}{N} \sum_{i=1}^N (x_i - \mu)^2}$

The following code replaces the mpg with a z-score. Cars with average MPG will be near zero, avobe zero is above average, and below zero is below average. Z-scores more that 3 above or below are very rare; these are outliers.

In [6]:
from scipy.stats import zscore

In [7]:
df = pd.read_csv("data/auto-mpg.csv", na_values=["NA", "?"])

df["mpg"] = zscore(df["mpg"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,-0.706439,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,-1.090751,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,-0.706439,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,-0.962647,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,-0.834543,8,302.0,140.0,3449,10.5,70,1,ford torino


### Encoding categorical values as Dummies
The traditional means of encoding categorical values is to make them dummy variables. This technique is also called `one-hot-encoding`. Consider the following data set. The area column is not numeric, so you must encode it with one-hot encoding. We display the number of areas and individual values. To show the number of categorical classes in the 'area' column:

In [2]:
df = pd.read_csv("data/jh-simple-dataset.csv", na_values=["NA", "?"])
df.head()

,id,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,1,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b
1,2,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c
2,3,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b
3,4,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b
4,5,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a


In [7]:
areas = list(df["area"].unique())
areas_sorted = sorted(areas)
print(f"Number of areas: {len(areas_sorted)}")
print(f"Areas: {areas_sorted}")

Number of areas: 4
Areas: ['a', 'b', 'c', 'd']


There are four unique values in the area column. To encode these dummy variables, we would use four columns, each representing one of the areas. For each row, one column would have a value of one, the rest zeros. For this reason , this type of encoding is sometimes called one-hot econding. The following code shows hot you might encode the values "a" through "d". The value A becomes [1,0,0,0] and the value B becomes [0,1,0,0]...

In [8]:
dummies = pd.get_dummies(areas_sorted, prefix="area", dtype=int)
print(dummies)

   area_a  area_b  area_c  area_d
0       1       0       0       0
1       0       1       0       0
2       0       0       1       0
3       0       0       0       1


In [9]:
# encode the categorical column of the dataframe named 'area'
dummies = pd.get_dummies(df["area"], prefix="area", dtype=int)
dummies.head()

,area_a,area_b,area_c,area_d
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,0,1


In [10]:
# merge the dummies into the original dataframe
df = pd.concat([df, dummies], axis=1)
df.head()

,id,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product,area_a,area_b,area_c,area_d
0,1,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b,0,0,1,0
1,2,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c,0,0,1,0
2,3,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b,0,0,1,0
3,4,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b,0,0,1,0
4,5,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a,0,0,0,1


In [11]:
# remove the original column 'area' because the goal is to get the data frame to be entirely numeric for the training
df.drop("area", axis=1, inplace=True)
df.head()

,id,job,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product,area_a,area_b,area_c,area_d
0,1,vv,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b,0,0,1,0
1,2,kd,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c,0,0,1,0
2,3,pe,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b,0,0,1,0
3,4,11,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b,0,0,1,0
4,5,kl,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a,0,0,0,1


### Encoding categorical values removing the first level of one-hot encoding
The pd.concat function also includes a parameter named drop_first, which specifies whether to get k-1 dummies out of k categorical levels by removing the first level. Why would you want to remove the first level, in this case, area_a? This technique provides a more efficient encoding by using the ordinarily unused encoding [0,0,0]. We encode the area to just three columns and map the categorical value of a to [0,0,0]. The following code demonstrates this technique.

In [13]:
df = pd.read_csv("data/jh-simple-dataset.csv", na_values=["NA", "?"])

# encode the area column as dummy variables dropping the first one-hot encoding category column
dummies = pd.get_dummies(df["area"], drop_first=True, prefix="area", dtype=int)
df = pd.concat([df, dummies], axis=1)
df.drop("area", axis=1, inplace=True)
df.head()

,id,job,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product,area_b,area_c,area_d
0,1,vv,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b,0,1,0
1,2,kd,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c,0,1,0
2,3,pe,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b,0,1,0
3,4,11,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b,0,1,0
4,5,kl,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a,0,0,1


### Target encoding for cetegoricals
Target encodign can somethimes increases the predictive power of a machine learning model. However, it also dramatically increases the risk of overfitting. This method must be used with care.

Generally, target encoding can only be used on a categorical feature when the output of the machine learning model is numeric (regression).

The concept of target encoding is straightforward. For each category, we calculate the average target value for that category. Then to encode, we substitute the percent corresponding to the category hat that the categorical value has. Unlike dummy variables, where you have a column for each category with target encoding, the program only needs a simgle column. In this way, target coding is more efficient than dummy variables.

In [15]:
np.random.seed(43)
df = pd.DataFrame(
    {
        "cont_9": np.random.rand(10)*100,
        "cat_0": ["dog"]*5 + ["cat"]*5,
        "cat_1": ["wolf"]*9 + ["tiger"]*1,
        "y": [1,0,1,1,1,1,0,0,0,0]
    }
)

df.head()

,cont_9,cat_0,cat_1,y
0,11.505457,dog,wolf,1
1,60.906654,dog,wolf,0
2,13.339096,dog,wolf,1
3,24.058962,dog,wolf,1
4,32.713906,dog,wolf,1
